In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import kornia 

In [3]:
from depth_decoder import DepthDecoder
from resnet_encoder import ResnetEncoder

encoder = ResnetEncoder(50)
decoder = DepthDecoder(encoder.num_ch_enc)

In [27]:
from kitti_dataset import KittiStereoDataset

data_dir = "/data-disk/kitti_data/"
train_file = open("filenames/kitti_train_files.txt", "r")
train_filenames = train_file.readlines()

train_dataset = KittiStereoDataset(data_dir, train_filenames, True)
batch_size = 2
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

image_l, image_r = next(iter(train_dataloader))

In [28]:
def test_loss(image_l, image_r):
    output_l = decoder(encoder(image_l))
    output_r = decoder(encoder(image_r))
    
    return (output_l, output_r)

In [29]:
l, r = test_loss(image_l, image_r)

In [30]:
l[('disp', 0)].size()

torch.Size([2, 1, 192, 640])

In [91]:
from loss import StereoDepthLoss, disparity_to_depth
import numpy as np

In [92]:
K = np.array([[718.856 ,   0.    , 607.1928],
               [  0.    , 718.856 , 185.2157],
               [  0.    ,   0.    ,   1.    ]])
b = 0.5372

batch_loss = StereoDepthLoss(K, b, 192, 640, batch_size)

In [93]:
output_l = l[('disp', 0)]
output_r = r[('disp', 0)]

batch_loss.calculate_loss((image_l.double(), image_r.double()), (output_l, output_r))

tensor(27.6648, dtype=torch.float64, grad_fn=<AddBackward0>)

In [90]:
promoted_type = torch.promote_types(image_l.dtype, output_l.dtype)

torch.float32
